In [11]:
import numpy as np
import scipy.linalg
import scipy.sparse

In [12]:
def basis_for_nullspace_of_dense_SVD(constr):
    '''Derives a matrix representing a basis for the nullspace of a dense matrix using SVD'''
    result = scipy.linalg.null_space(constr)
    # Scaling: Largest entry should be 1
    for i in range(result.shape[1]):
        mx = result[:,i].max()
        mn = result[:,i].min()
        if mx > -mx:
            result[:,i] /= mx
        else:
            result[:,i] /= mn
    return result.transpose()

def basis_for_nullspace_of_dense_LU(constr, tol=1e-7):
    '''Derives a matrix representing a basis for the nullspace of a dense matrix using LU'''
    P,L,U = scipy.linalg.lu(constr)
    r,c = U.shape
    # Remove (almost) empty rows
    for i in range(r-1,-1,-1):
        if (abs(U[i,:])<tol).all():
            U = np.delete(U,i,0)
    r,c = U.shape
    # Swap cols if necessary
    perm = np.arange(c)
    for i in range(r):
        if abs(U[i,i])<tol:
            mx = np.abs(U[i,i+1:]).max()
            assert mx > tol
            j = i
            while abs(U[i,j]) < mx:
                j += 1
            perm[i], perm[j] = perm[j], perm[i]
            U[:,[i,j]] = U[:,[j,i]]
    # Split matrix U and resolve linear syste
    U1 = U[:,:r]
    U2 = U[:,r:]
    print(U1)
    print(U2)
    sol = scipy.linalg.lu_solve((U1,np.arange(r)),U2)
    # Setup result
    result = np.zeros((c,c-r))
    result[:r,:] = -sol
    result[r:,:] = np.eye(c-r)    
    # Apply perm
    result = result[perm.tolist()]
    # Scaling: Largest entry should be 1
    for i in range(result.shape[1]):
        mx = result[:,i].max()
        mn = result[:,i].min()
        if mx > -mx:
            result[:,i] /= mx
        else:
            result[:,i] /= mn

    return result.transpose()

In [13]:
def get_connected_components(constr):
    '''Derives all connected components of the graph, where cols of matrix are nodes'''
    assert isinstance(constr, scipy.sparse.csr_matrix)
    (r, c) = constr.shape
    
    if not constr.has_sorted_indices:
        constr.sort_indices()
    
    comps = np.arange(c)
    for i in range(r):
        smallest_index = c
        for j in range(constr.indptr[i], constr.indptr[i+1]):
            smallest_index = min(smallest_index, comps[constr.indices[j]])
        for j in range(constr.indptr[i], constr.indptr[i+1]):
            comps[constr.indices[j]] = smallest_index        
    
    for i in range(c):
        if comps[comps[i]] < comps[i]:
            comps[i] = comps[comps[i]]
        
    constr_list = {}
    for i in range(r):
        comp = comps[constr.indices[constr.indptr[i]]]
        if comp not in constr_list:
            constr_list[comp] = []         
        constr_list[comp].append(i)
        
    comp_list = {}
    coupled = np.zeros(c,'b')
    for i in range(c):
        comp = comps[i]
        if comp != i or comp in constr_list.keys():
            if not comp in comp_list:
                comp_list[comp] = [comp]
            if comp != i:
                comp_list[comp].append(i)
            coupled[i] = True
    
    #print("coupled     =",coupled)
    #print("comp_list   =",comp_list)
    #print("constr_list =",constr_list)
    
    return coupled, comp_list, constr_list

In [14]:
def basis_for_nullspace(constr, basis_for_nullspace_of_dense=basis_for_nullspace_of_dense_SVD):
    '''Derives a matrix representing a basis for the nullspace of a sparse matrix by first decomposing it into its connected components'''
    assert isinstance(constr, scipy.sparse.csr_matrix)
    (r, c) = constr.shape
    
    coupled, comp_list, constr_list = get_connected_components(constr)

    mat = scipy.sparse.lil_matrix((c,c))
    
    # First, take the free dofs as they are
    row = 0
    for i in range(c):
        if not coupled[i]:
            mat[row,i] = 1
            row += 1

    # TODO: Secondly, take the dofs with one-to-one mapping
    
    # Finally, take the other dofs
    for i in comp_list.keys():
        comp_item = comp_list[i]
        constr_item = constr_list[i]
        local_matrix = (constr[constr_item,:][:,comp_item]).todense()
        local_basis  = basis_for_nullspace_of_dense(local_matrix)
        #print( "local_matrix:\n", local_matrix )
        #print( "local_basis :\n", local_basis  )
        lb_rows, lb_cols = local_basis.shape
        for i0 in range(lb_rows):
            for j0 in range(lb_cols):
                mat[row,comp_item[j0]] = local_basis[i0,j0]
            row += 1

    mat.resize(row,c)
    return mat.tocsr()

In [15]:
def run_testcases(constr):
    print(constr.todense())
    
    mat = basis_for_nullspace_of_dense_SVD(constr.todense())
    print("Result based on SVD globally:\n",mat)
    assert (np.linalg.norm(constr.todense() @ mat.transpose()) < 1e-7)

    mat = basis_for_nullspace(constr).todense()
    print("Result based on SVD locally:\n",mat)
    assert (np.linalg.norm(constr.todense() @ mat.transpose()) < 1e-7)
    
    mat = basis_for_nullspace_of_dense_LU(constr.todense())
    print("Result based on LU globally:\n",mat)
    assert (np.linalg.norm(constr.todense() @ mat.transpose()) < 1e-7)
    
    mat = basis_for_nullspace(constr, basis_for_nullspace_of_dense_LU).todense()
    print("Result based on LU locally:\n",mat)
    assert (np.linalg.norm(constr.todense() @ mat.transpose()) < 1e-7)

In [16]:
constr = scipy.sparse.csr_matrix([[1.,-1,0,0,0,0],[1,0,-1,0,0,0],[0,1,-1,0,0,0],[0,0,0,0,0,1]])
run_testcases(constr)

[[ 1. -1.  0.  0.  0.  0.]
 [ 1.  0. -1.  0.  0.  0.]
 [ 0.  1. -1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.]]
Result based on SVD globally:
 [[ 1.  1.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.]
 [-0. -0. -0.  1. -0. -0.]]
Result based on SVD locally:
 [[0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [1. 1. 1. 0. 0. 0.]]
[[ 1. -1.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]]
[[ 0.  0.  0.]
 [ 0.  0. -1.]
 [ 0.  0.  0.]]
Result based on LU globally:
 [[-0. -0.  0.  1.  0. -0.]
 [-0. -0.  0.  0.  1. -0.]
 [ 1.  1.  1.  0.  0. -0.]]
[[ 1. -1.]
 [ 0.  1.]]
[[ 0.]
 [-1.]]
[[1.]]
[]
Result based on LU locally:
 [[0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [1. 1. 1. 0. 0. 0.]]


In [17]:
constr = scipy.sparse.csr_matrix([[1.,-1,0,0,0,0],[0,1,-1,0,0,0],[0,0,0,0,-1,1]])
run_testcases(constr)

[[ 1. -1.  0.  0.  0.  0.]
 [ 0.  1. -1.  0.  0.  0.]
 [ 0.  0.  0.  0. -1.  1.]]
Result based on SVD globally:
 [[ 0.          0.          0.          1.          0.          0.        ]
 [ 0.81649658  0.81649658  0.81649658  0.          1.          1.        ]
 [-0.81649658 -0.81649658 -0.81649658  0.          1.          1.        ]]
Result based on SVD locally:
 [[0. 0. 0. 1. 0. 0.]
 [1. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1.]]
[[ 1. -1.  0.]
 [ 0.  1.  0.]
 [ 0.  0. -1.]]
[[ 0.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0.  1.]]
Result based on LU globally:
 [[-0. -0.  0.  1.  0.  0.]
 [ 1.  1.  1.  0.  0.  0.]
 [-0. -0.  0.  0.  1.  1.]]
[[ 1. -1.]
 [ 0.  1.]]
[[ 0.]
 [-1.]]
[[-1.]]
[[1.]]
Result based on LU locally:
 [[0. 0. 0. 1. 0. 0.]
 [1. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1.]]


In [18]:
constr = scipy.sparse.csr_matrix([
     [   1,  0,   -1, 0, 0, 0],
     [   0,  1,    0, 0, 0,-1],
     [  .5, .5,    0,-1, 0, 0],
     [  .5, .5,    0, 0,-1, 0],
])
run_testcases(constr)

[[ 1.   0.  -1.   0.   0.   0. ]
 [ 0.   1.   0.   0.   0.  -1. ]
 [ 0.5  0.5  0.  -1.   0.   0. ]
 [ 0.5  0.5  0.   0.  -1.   0. ]]
Result based on SVD globally:
 [[ 1.          0.7313967   1.          0.86569835  0.86569835  0.7313967 ]
 [-0.81253903  1.         -0.81253903  0.09373049  0.09373049  1.        ]]
Result based on SVD locally:
 [[ 1.          0.7313967   1.          0.86569835  0.86569835  0.7313967 ]
 [-0.81253903  1.         -0.81253903  0.09373049  0.09373049  1.        ]]
[[ 1.   0.  -1.   0. ]
 [ 0.   1.   0.   0. ]
 [ 0.   0.   0.5 -1. ]
 [ 0.   0.   0.   1. ]]
[[ 0.   0. ]
 [ 0.  -1. ]
 [ 0.   0.5]
 [-1.   0. ]]
Result based on LU globally:
 [[ 1.  -0.   1.   0.5  0.5  0. ]
 [-1.   1.  -1.  -0.   0.   1. ]]
[[ 1.   0.  -1.   0. ]
 [ 0.   1.   0.   0. ]
 [ 0.   0.   0.5 -1. ]
 [ 0.   0.   0.   1. ]]
[[ 0.   0. ]
 [ 0.  -1. ]
 [ 0.   0.5]
 [-1.   0. ]]
Result based on LU locally:
 [[ 1.   0.   1.   0.5  0.5  0. ]
 [-1.   1.  -1.   0.   0.   1. ]]


In [19]:
constr = scipy.sparse.csr_matrix([
     [   1,  0,   -1, 0, 0, 0],
     [   0,  1,    0, 0, 0,-1],
     [  .5, .5,    0,-1, 0, 0],
     [   0,  0,    0, 1,-1, 0],
])
run_testcases(constr)

[[ 1.   0.  -1.   0.   0.   0. ]
 [ 0.   1.   0.   0.   0.  -1. ]
 [ 0.5  0.5  0.  -1.   0.   0. ]
 [ 0.   0.   0.   1.  -1.   0. ]]
Result based on SVD globally:
 [[ 1.          0.2199828   1.          0.6099914   0.6099914   0.2199828 ]
 [-0.40228371  1.         -0.40228371  0.29885815  0.29885815  1.        ]]
Result based on SVD locally:
 [[ 1.          0.2199828   1.          0.6099914   0.6099914   0.2199828 ]
 [-0.40228371  1.         -0.40228371  0.29885815  0.29885815  1.        ]]
[[ 1.   0.  -1.   0. ]
 [ 0.   1.   0.   0. ]
 [ 0.   0.   0.5 -1. ]
 [ 0.   0.   0.   1. ]]
[[ 0.   0. ]
 [ 0.  -1. ]
 [ 0.   0.5]
 [-1.   0. ]]
Result based on LU globally:
 [[ 1.  -0.   1.   0.5  0.5  0. ]
 [-1.   1.  -1.  -0.   0.   1. ]]
[[ 1.   0.  -1.   0. ]
 [ 0.   1.   0.   0. ]
 [ 0.   0.   0.5 -1. ]
 [ 0.   0.   0.   1. ]]
[[ 0.   0. ]
 [ 0.  -1. ]
 [ 0.   0.5]
 [-1.   0. ]]
Result based on LU locally:
 [[ 1.   0.   1.   0.5  0.5  0. ]
 [-1.   1.  -1.   0.   0.   1. ]]


In [20]:
constr = scipy.sparse.csr_matrix([
     [   1,  0,   -1, 0, 0, 0],
     [   0,  1,    0, 0, 0,-1],
     [  .5, .5,    0,-1, 0, 0],
     [  .5, .5,    0, 0,-1, 0],
     [   0,  0,    0, 1,-1, 0],
])
run_testcases(constr)

[[ 1.   0.  -1.   0.   0.   0. ]
 [ 0.   1.   0.   0.   0.  -1. ]
 [ 0.5  0.5  0.  -1.   0.   0. ]
 [ 0.5  0.5  0.   0.  -1.   0. ]
 [ 0.   0.   0.   1.  -1.   0. ]]
Result based on SVD globally:
 [[ 1.          0.9849657   1.          0.99248285  0.99248285  0.9849657 ]
 [-0.98995196  1.         -0.98995196  0.00502402  0.00502402  1.        ]]
Result based on SVD locally:
 [[ 1.          0.9849657   1.          0.99248285  0.99248285  0.9849657 ]
 [-0.98995196  1.         -0.98995196  0.00502402  0.00502402  1.        ]]
[[ 1.   0.  -1.   0. ]
 [ 0.   1.   0.   0. ]
 [ 0.   0.   0.5 -1. ]
 [ 0.   0.   0.   1. ]]
[[ 0.   0. ]
 [ 0.  -1. ]
 [ 0.   0.5]
 [-1.   0. ]]
Result based on LU globally:
 [[ 1.  -0.   1.   0.5  0.5  0. ]
 [-1.   1.  -1.  -0.   0.   1. ]]
[[ 1.   0.  -1.   0. ]
 [ 0.   1.   0.   0. ]
 [ 0.   0.   0.5 -1. ]
 [ 0.   0.   0.   1. ]]
[[ 0.   0. ]
 [ 0.  -1. ]
 [ 0.   0.5]
 [-1.   0. ]]
Result based on LU locally:
 [[ 1.   0.   1.   0.5  0.5  0. ]
 [-1.   1.  -1.   0.

In [21]:
#test case: T-junction itself
#0 and 1 are coarse dofs, 2,3 and 4,5 are fine dofs and 3 and 4 are also joined at the T-junction itself.
constr = scipy.sparse.csr_matrix([
     [   1,  0,  0, -1,  0,  0,  0,  0,  0],
     [  .5, .5,  0,  0, -1,  0,  0,  0,  0],
     [   0,  1,  0,  0,  0, -1,  0,  0,  0],
     [   0,  1,  0,  0,  0,  0, -1,  0,  0],
     [   0, .5, .5,  0,  0,  0,  0, -1,  0],
     [   0,  0, -1,  0,  0,  0,  0,  0, -1],
     [   0,  0,  0,  0,  0,  1, -1,  0,  0]
])
#run_testcases(constr)

In [22]:
#mat = basis_for_nullspace_of_dense_LU(constr.A)
mat = basis_for_nullspace(constr, basis_for_nullspace_of_dense_LU)

[[ 1.   0.   0.  -1.   0.   0. ]
 [ 0.   1.   0.   0.   0.  -1. ]
 [ 0.   0.  -1.   0.   0.   0. ]
 [ 0.   0.   0.   0.5  0.   0.5]
 [ 0.   0.   0.   0.  -1.   0.5]
 [ 0.   0.   0.   0.   0.   1. ]]
[[ 0.   0.   0. ]
 [ 0.   0.   0. ]
 [ 0.   0.  -1. ]
 [ 0.  -1.   0. ]
 [ 0.   0.  -0.5]
 [-1.   0.   0. ]]


In [23]:
mat.A

array([[-1. ,  1. ,  0. , -1. ,  0. ,  1. ,  1. ,  0.5,  0. ],
       [ 1. ,  0. ,  0. ,  1. ,  0.5,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. , -0.5,  1. ]])

In [24]:
P,L,U = scipy.linalg.lu(constr.A)

In [25]:
U

array([[ 1. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  1. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. , -1. ],
       [ 0. ,  0. ,  0. ,  0.5, -1. ,  0.5,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0.5,  0. , -1. , -0.5],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  1. , -1. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ]])

In [26]:
U

array([[ 1. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  1. ,  0. ,  0. ,  0. , -1. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. , -1. ,  0. ,  0. ,  0. ,  0. ,  0. , -1. ],
       [ 0. ,  0. ,  0. ,  0.5, -1. ,  0.5,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0.5,  0. , -1. , -0.5],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  1. , -1. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ]])

NameError: name 'sol' is not defined